# Predict Game Output in League Of Legends
---
**Por: Ian Joab Padron Corona - A01708940**

## Librerias

In [ ]:
'''
===============================================================================
Librerias de Python a utilizar en el proyecto
===============================================================================
pandas: Libreria de manipulacion de datos en DataFrames
tensorflow: Libreria de aprendizaje profundo
keras: API de alto nivel para crear redes neuronales
Sequential: API para crear modelos de redes neuronales
Dense: Capa densa (fully connected)
Conv2D: Capa de convolucion 2D
MaxPooling2D: Capa de max pooling 2D
OneHotEncoder: Convertir variables categoricas en numericas creando columnas binarias. ESTE SOLO PARA EL DE TETRIS
MinMaxScaler: Normaliza los datos entre 0 y 1
train_test_split: Divide los datos en conjuntos de entrenamiento y prueba
===============================================================================
'''

import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

## Datos

In [ ]:
data = pd.read_csv('../content/high_diamond_ranked_10min.csv', low_memory=False)
data.drop(columns=['gameId'], inplace=True)

In [ ]:
print(data.info())

In [ ]:
data.boxplot(figsize = (16, 8))

## Escalamiento

In [ ]:
scaler = MinMaxScaler()

# Escalamiento de las columnas con valores muy grandes
columns_to_scale = ['blueTotalGold', 'blueTotalExperience', 'blueGoldDiff', 'blueExperienceDiff', 'blueGoldPerMin',
                    'redTotalGold', 'redTotalExperience', 'redGoldDiff', 'redExperienceDiff', 'redGoldPerMin']
data_scaled = data.copy()
data_scaled[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

In [ ]:
data_scaled.boxplot(figsize = (16, 8))

## Separacion

In [ ]:
# Asignar las 'features; X' y la 'target variable; y'
X = data_scaled.drop(columns=['blueWins'])
y = data_scaled['blueWins']

# Dividir los datos en en train y test
X_train, X_test, y_train, y_test = train_test_split(
                                                    X,
                                                    y,
                                                    train_size   = 0.80,
                                                    random_state = 42,
                                                    shuffle      = True
                                                    )

## Modelo

### Classifier

In [ ]:
def model_simple(shape):
    """
    This is a simple model multilayer perceptron or neural network.
    Your function should return the model.
    """
    model  =  Sequential([
                    Dense(128, activation='relu', input_shape=shape),
                    Dense(128, activation='relu'),
                    Dense(1, activation='sigmoid')
            ])
    return model

In [ ]:
def compile_model_simple(model):
    """
    This function takes in the model returned from your get_model function, and compiles it with an optimiser,
    loss function and metric.
    Compile the model using the Adam optimiser (with default settings), the cross-entropy loss function and
    accuracy as the only metric.
    Your function doesn't need to return anything; the model will be compiled in-place.
    """
    model.compile(optimizer= "adam",
                  loss = "binary_crossentropy",
                  metrics=['accuracy'])

In [ ]:
def train_model_simple(model, train_features, train_labels):
    """
    Train the model on the scaled_train_images and train_labels.
    Your function should return the training history, as returned by model.fit.
    """
    return model.fit(train_features, train_labels, epochs = 20)

In [ ]:
model_nn = model_simple(X_train.shape[1:])
model_nn.summary()

In [ ]:
compile_model_simple(model_nn)

In [ ]:
history_nn = train_model_simple(model_nn, X_train, y_train)

### Plot accuracy

In [ ]:
frame = pd.DataFrame(history_nn.history)

In [ ]:
# Run this cell to make the Accuracy vs Epochs plot

acc_plot = frame.plot(y="accuracy", title="Accuracy vs Epochs", legend=False)
acc_plot.set(xlabel="Epochs", ylabel="Accuracy")

In [ ]:
# Run this cell to make the Loss vs Epochs plot

acc_plot = frame.plot(y="loss", title = "Loss vs Epochs",legend=False)
acc_plot.set(xlabel="Epochs", ylabel="Loss")

### Evaluacion

In [ ]:
def evaluate_model(model, test_features, test_labels):
    """
    This function should evaluate the model on the scaled_test_images and test_labels.
    Your function should return a tuple (test_loss, test_accuracy).
    """
    test_loss, test_accuracy = model.evaluate(test_features, test_labels)
    return (test_loss, test_accuracy)

In [ ]:
# Run your function to evaluate the model Neural Network Dense Layers

test_loss, test_accuracy = evaluate_model(model_nn, X_test, y_test)
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")